### Instalação dos pacotes em Julia

In [ ]:
import Pkg; Pkg.add("JuMP")
import Pkg; Pkg.add("HiGHS")
import Pkg; Pkg.add("GLPK")
import Pkg; Pkg.add("Clp")

### Importando às bibliotecas que serão usadas para resolver os problemas de PLI

In [2]:
using JuMP
import HiGHS
import GLPK
using Clp

## Lista 1

### Questão 1

In [35]:
#model = Model(HiGHS.Optimizer)
model = Model(GLPK.Optimizer)

i = 2 # Número de Usinas
j = 3 # Número de Clientes

weights = [1, 1.5, 3.5, 2, 1, 2]

@variable(model, x[1:i, 1:j] >= 0)
# As demandas respectivas para o produto são:
@constraint(model, x[1,1] + x[2,1]   == 100) #E_1
@constraint(model, x[1,2] + x[2,2]  == 200) # E_2
@constraint(model, x[1,3] + x[2,3]  == 300) # E_3
# Restições de Capacidade
@constraint(model, x[1,1] + x[1,2] + x[1,3] <= 400) # U_1
@constraint(model, x[2,1] + x[2,2] + x[2,3] <= 300) # U_2

# Função Objetivo
@objective(model, Min, sum(weights[n] * x[div(n-1,3)+1, mod(n-1,3)+1] for n in 1:length(weights)))

optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))

for m in 1:i
    for n in 1:j
        println("x[$m, $n] = ", round(Int, value(x[m,n])))
    end
end

Valor Objetivo (Custo Mínimo): 1000.0
x[1, 1] = 100
x[1, 2] = 200
x[1, 3] = 0
x[2, 1] = 0
x[2, 2] = 0
x[2, 3] = 300


Função Objetivo: $x_{1,1} + 1.5x_{1,2} + 3.5x_{1,3} + 2x_{2,1} + x_{2,2} + 2x_{2,3}$

Generalizando para todos os casos na forma:

$$ \begin{align} \min \; & c^T x \\
s.t \; & Ax = b \\
& Cx \leq d \\
& x \geq 0 
\end{align}$$

In [74]:
function LPP_Transporte(i, j, weights, A, C, b, d)
    model = Model(GLPK.Optimizer)
    p = length(weights)
    @variable(model, x[1:i, 1:j] >= 0)
    x = reshape(x', p, 1)
    for m in 1:length(b)
        @constraint(model, (A*x)[m] == b[m])
    end
    for n in 1:length(d)
        @constraint(model, (C*x)[n] <= d[n])
    end
    @objective(model, Min, sum(weights[k] * x[k] for k in 1:p))
    println("Função Objetivo: ", objective_function(model))
    optimize!(model)
    for k in 1:p
        #m,n = div(k-1, j) + 1, mod(k-1, j) + 1
        println(x[k]," = ", value(x[k]))
    end
    println("Valor Objetivo (Custo Mínimo): ", objective_value(model))
    #print(x)
end

# Printa o valor das variáveis
function print_var(x)
    if typeof(x) == Matrix{VariableRef}
        for i in 1:size(x,1)
            for j in 1:size(x,2)
                if value(x[i,j]) - round(value(x[i,j])) < 1e-3
                    println(x[i,j], " = ", round(Int, value(x[i,j])))
                else
                    println(x[i,j], " = ", value(x[i,j]))
                end
            end
        end
    elseif typeof(x) == Vector{VariableRef}
        for i in 1:length(x)
            println(x[i]," = ", value(x[i]))
        end
    elseif typeof(x) == VariableRef
        println(x, " = ", value(x))
    else 
        println("Tipo de variável não suportado")
    end
end

print_var (generic function with 1 method)

Exemplificando com o problema anterior...

In [62]:
A = [1 0 0 1 0 0;
0 1 0 0 1 0;
0 0 1 0 0 1]
b = [100;200;300]

C = [1 1 1 0 0 0;
 0 0 0 1 1 1]
d = [400; 300]

weights = [1 1.5 3.5 2 1 2]
i = 2
j = 3

LPP_Transporte(i, j, weights, A, C, b, d)

Função Objetivo: x[1,1] + 1.5 x[1,2] + 3.5 x[1,3] + 2 x[2,1] + x[2,2] + 2 x[2,3]
x[1,1] = 100.0
x[1,2] = 200.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 300.0
Valor Objetivo (Custo Mínimo): 1000.0


### Questão 2

Item 1.

In [96]:
model = Model(HiGHS.Optimizer)
# Quantidades produzidas pela empresa do produto I e II
@variable(model, p[1:i] >= 0)
# Função Objetivo
@objective(model, Max, 7*p[1] + 9*p[2])
# Restrição de Recurso
@constraint(model, p[1] + p[2] <= 8)
@constraint(model, 2*p[1] + 3*p[2] <= 4)
@constraint(model, p[2] <= 19)

set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))

for m in 1:2
    println("p[$m] = ", round(Int, value(p[m])))
end

Valor Objetivo (Custo Mínimo): 14.0
p[1] = 2
p[2] = 0


Item 2.

In [101]:
model = Model(HiGHS.Optimizer)
# Preços de 1 KG de NO3, KCL, PO4
@variable(model, x[1:j] >= 0)
# Função Objetivo
@objective(model, Min, 9*x[1] + 19*x[2] + 4*x[3])
# Restrição de Lucro
@constraint(model, 9*x[1] + 2*x[2] >= 7)
@constraint(model, x[1] + 3*x[2] + x[3] >= 9)

set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))

for m in 1:3
    println("p[$m] = ", round(Int, value(x[m])))
end

Valor Objetivo (Custo Mínimo): 39.888888888888886
p[1] = 1
p[2] = 0
p[3] = 8


### Questão 3. [Transporte] 

Um modelo de carro é montado em 3 usinas situadas nas cidades \(V1\), \(V2\) e \(V3\). O motor destes modelos é fornecido por duas outras usinas situadas nas cidades \(U1\) e \(U2\). As usinas de montagem precisam de pelo menos 5, 4 e 3 motores, respectivamente. Cada usina pode fornecer no máximo 6 motores. A direção da empresa quer minimizar o custo de transporte dos motores entre os dois sítios de fábrica e os três sítios de montagem. Os custos unitários (por motor transportado) para todos os itinerários possíveis são:


\begin{array}{|c|c|c|c|}
\hline
 & \text{V1} & \text{V2} & \text{V3} \\
\hline
\text{U1} & 38 & 27 & 48 \\
\hline
\text{U2} & 37 & 58 & 45 \\
\hline
\end{array}

Como minimizar o custo total de transporte respeitando a oferta e a demanda?



In [3]:
# Define os valores de oferta e demanda
oferta = [6, 6]
demanda = [5, 4, 3]

# Define os custos de transporte
custos = [38 27 48;
          37 58 45]

# Cria um modelo JuMP
modelo = Model(GLPK.Optimizer)

# Define as variáveis de decisão
@variable(modelo, x[1:2, 1:3] >= 0)

# Define as restrições de oferta
for i in 1:2
    @constraint(modelo, sum(x[i, j] for j in 1:3) <= oferta[i])
end

# Define as restrições de demanda
for j in 1:3
    @constraint(modelo, sum(x[i, j] for i in 1:2) >= demanda[j])
end

# Define a função objetivo para minimizar o custo total de transporte
@objective(modelo, Min, sum(custos[i, j] * x[i, j] for i in 1:2, j in 1:3))

# Resolve o problema de otimização
optimize!(modelo)

# Verifica se o problema é resolvido de forma ótima
@assert termination_status(modelo) == MOI.OPTIMAL

println(modelo)

println("Solução Ótima:")
for i in 1:2
    for j in 1:3
        println("x[$i, $j] = ", value(x[i, j]))
    end
end

println("Custo Total de Transporte: ", objective_value(modelo))

Min 38 x[1,1] + 27 x[1,2] + 48 x[1,3] + 37 x[2,1] + 58 x[2,2] + 45 x[2,3]
Subject to
 x[1,1] + x[2,1] >= 5
 x[1,2] + x[2,2] >= 4
 x[1,3] + x[2,3] >= 3
 x[1,1] + x[1,2] + x[1,3] <= 6
 x[2,1] + x[2,2] + x[2,3] <= 6
 x[1,1] >= 0
 x[2,1] >= 0
 x[1,2] >= 0
 x[2,2] >= 0
 x[1,3] >= 0
 x[2,3] >= 0

Solução Ótima:
x[1, 1] = 2.0
x[1, 2] = 4.0
x[1, 3] = 0.0
x[2, 1] = 3.0
x[2, 2] = 0.0
x[2, 3] = 3.0
Custo Total de Transporte: 430.0


### Questão 4

### Questão 5

### Questão 6

In [75]:
model = Model(HiGHS.Optimizer)

i = 4
j = 3
# Quantidade do tipo i por dia pro Diesel j (1 -> A, 2 -> B, 3 -> C)
@variable(model, Q[1:i, 1:j] >= 0)

# Restrição da quantidade máxima disposta por dia de cada tipo de petróleo
@constraint(model, sum(Q[1,n] for n in 1:j) <= 3000)
@constraint(model, sum(Q[2,n] for n in 1:j) <= 2000)
@constraint(model, sum(Q[3,n] for n in 1:j) <= 4000)
@constraint(model, sum(Q[4,n] for n in 1:j) <= 1000)


# Restrições de quantidade de cada tipo de petróleo no Diesel A
@constraint(model, Q[1,1] <= 0.3 * sum(Q[m,1] for m in 1:i))
@constraint(model, Q[2,1] <= 0.5 * sum(Q[m,1] for m in 1:i))
@constraint(model, Q[3,1] <= 0.4 * sum(Q[m,1] for m in 1:i))

# Restrições de quantidade de cada tipo de petróleo no Diesel B
@constraint(model, Q[1,2] <= 0.5 * sum(Q[m,2] for m in 1:i))
@constraint(model, Q[2,2] <= 0.1 * sum(Q[m,2] for m in 1:i))

# Restrições de quantidade de cada tipo de petróleo no Diesel C
@constraint(model, Q[1,3] <= 0.7 * sum(Q[m,3] for m in 1:i))

# Função Objetivo
@objective(model, Max, 5.5 * sum(Q[m,1] for m in 1:i) + 4.5 * sum(Q[m,2] for m in 1:i) + 3.5 * sum(Q[m,3] for m in 1:i) - 3*(Q[1,1] + Q[1,2] + Q[1,3]) - 6*(Q[2,1] + Q[2,2] + Q[2,3]) - 4*(Q[3,1] + Q[3,2] + Q[3,3]) - 5*(Q[4,1] + Q[4,2] + Q[4,3]))

2.5 Q[1,1] - 0.5 Q[2,1] + 1.5 Q[3,1] + 0.5 Q[4,1] + 1.5 Q[1,2] - 1.5 Q[2,2] + 0.5 Q[3,2] - 0.5 Q[4,2] + 0.5 Q[1,3] - 2.5 Q[2,3] - 0.5 Q[3,3] - 1.5 Q[4,3]

In [76]:
set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
print("Valor Objetivo (Lucro Máximo): ", objective_value(model))
print_var(Q)

Valor Objetivo (Lucro Máximo): 13000.0Q[1,1] = 3000
Q[1,2] = 0
Q[1,3] = 0
Q[2,1] = 2000
Q[2,2] = 0
Q[2,3] = 0
Q[3,1] = 4000
Q[3,2] = 0
Q[3,3] = 0
Q[4,1] = 1000
Q[4,2] = 0
Q[4,3] = 0


## Lista 2

### Problema da Mochila

In [77]:
n = 5; # Número de Itens
capacity = 10.0; # Capacidade da Mochila
profit = [5.0, 3.0, 2.0, 7.0, 4.0]; # Lucro de cada item
weight = [2.0, 8.0, 4.0, 2.0, 5.0]; # Peso de cada item

#model = Model(GLPK.Optimizer)
model = Model(HiGHS.Optimizer)

# Variáveis Binárias
@variable(model, x[1:n], Bin)
# Restrição de Capacidade
@constraint(model, sum(weight[i] * x[i] for i in 1:n) <= capacity)
# Função Objetivo
@objective(model, Max, sum(profit[i] * x[i] for i in 1:n))
print(model) # LPP

Max 5 x[1] + 3 x[2] + 2 x[3] + 7 x[4] + 4 x[5]
Subject to
 2 x[1] + 8 x[2] + 4 x[3] + 2 x[4] + 5 x[5] <= 10
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary


In [78]:
set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))
print("Itens Escolhidos: ", [i for i in 1:n if value(x[i]) == 1])

Valor Objetivo (Custo Mínimo): 16.0
Itens Escolhidos: [1, 4, 5]

## Prova A1

### Questão 1 

In [79]:
model = Model(HiGHS.Optimizer)

i = 4
j = 3

# l_i: quantidade de líquido i revendido diretamente
@variable(model, l[1:i] >= 0)
# x_ij: quantidade de líquido i = 1,2,3,4 (A,B,C,D) usado na produção de j = 1,2,3 (E,F,G)
@variable(model, x[1:i, 1:j] >= 0)
# p: quantidade de líquido P produzido
@variable(model, p >= 0)

# make sum of x[m,j] <= 8000 as constraint
d = [8000, 4250, 16000, 2000]
for m in 1:i
    @constraint(model, l[m] + sum(x[m,j] for j in 1:j) <= d[m])
end

# Para produzir E, j=1
@constraint(model, x[1,1] == 0.3 * sum(x[m,1] for m in 1:i)) # A 
@constraint(model, x[2,1] >= 0.1 * sum(x[m,1] for m in 1:i)) # B
@constraint(model, x[3,1] == 0.4 * sum(x[m,1] for m in 1:i)) # C
@constraint(model, x[4,1] <= 0.05 * sum(x[m,1] for m in 1:i)) # D

# Para produzir F, j=2
@constraint(model, x[1,2] == 0.25 * sum(x[m,2] for m in 1:i)) # A
@constraint(model, x[2,2] <= 0.2 * sum(x[m,2] for m in 1:i)) # B
@constraint(model, x[3,2] == 0.2 * sum(x[m,2] for m in 1:i)) # C
@constraint(model, x[4,2] >= 0.1 * sum(x[m,2] for m in 1:i)) # D

# Para produzir G, j=3
@constraint(model, x[1,3] == 0.2 * sum(x[m,3] for m in 1:i)) # A
@constraint(model, x[2,3] >= 0.15 * sum(x[m,3] for m in 1:i)) # B
@constraint(model, x[3,3] == 0.4 * sum(x[m,3] for m in 1:i)) # C
@constraint(model, x[4,3] <= 0.2 * sum(x[m,3] for m in 1:i)) # D

# A produção de líquidos E,F,G deve suprir a obrigação da empresa
@constraint(model, sum(x[m,1] for m in 1:i) >= 400)
@constraint(model, sum(x[m,2] for m in 1:i) >= 800)
@constraint(model, sum(x[m,3] for m in 1:i) >= 200)

# O líquido P deve satisfazer as proporções especificadas de líquido E e G
@constraint(model, p/3 <= sum(x[m,1] for m in 1:i))
@constraint(model, 2*p/3 <= sum(x[m,3] for m in 1:i))

# Função Objetivo
@objective(model, Max, 6*(sum(l[m] for m in 1:i)) + 11*(sum(x[m,1] for m in 1:i) - p/3) + 15*(sum(x[m,2] for m in 1:i)) + 14*(sum(x[m,3] for m in 1:i) - 2*p/3)) + 22*p - 5.5*(l[1] + sum(x[1,n] for n in 1:j)) - 3.5*(l[2] + sum(x[2,j] for j in 1:j)) - 7.5*(l[3] + sum(x[3,j] for j in 1:j)) - 11.5*(l[4] + sum(x[4,j] for j in 1:j));

In [86]:
set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
println("Valor Objetivo (Lucro Máximo): ", objective_value(model))
print_var(x)
print_var(p)
print_var(l)

Valor Objetivo (Lucro Máximo): 304500.0
x[1,1] = 120
x[1,2] = 200
x[1,3] = 2845
x[2,1] = 120
x[2,2] = 0
x[2,3] = 4130
x[3,1] = 160
x[3,2] = 160
x[3,3] = 5690
x[4,1] = 0
x[4,2] = 440
x[4,3] = 1560
p = 0.0
l[1] = 4835.0
l[2] = 0.0
l[3] = 9990.0
l[4] = 0.0


### Questão 2

In [87]:
model = Model(HiGHS.Optimizer)

i = 3
j = 3
weights = [50, 90, 80, 0, 30, 40, 0, 60, 50]

@variable(model, x[1:i, 1:j] >= 0)
# As demandas respectivas para cada categoria devem ser atendidas
@constraint(model, x[1,1] == 20) # Iluminação
@constraint(model, x[1,2] + x[2,2] + x[3,2] == 10) # Aquecimento Ambiente
@constraint(model, x[1,3] + x[2,3] + x[3,3]  == 30) # Aquecimento Água
# Restrições de Capacidade
@constraint(model, x[1,1] + x[1,2] + x[1,3]  <= 50) # Eletricidade
@constraint(model, x[2,2] + x[2,3]  <= 50) # Painéis Solares
@constraint(model, x[3,2] + x[3,3]  <= 20) # Gás Natural

# Função Objetivo
@objective(model, Min, sum(weights[n] * x[div(n-1,3)+1, mod(n-1,3)+1] for n in 1:9))

50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 30 x[2,2] + 40 x[2,3] + 60 x[3,2] + 50 x[3,3]

In [88]:
set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))

for m in 1:i
    for n in 1:j
        println("x[$m, $n] = ", round(Int, value(x[m,n])))
    end
end

Valor Objetivo (Custo Mínimo): 2500.0
x[1, 1] = 20
x[1, 2] = 0
x[1, 3] = 0
x[2, 1] = 0
x[2, 2] = 10
x[2, 3] = 30
x[3, 1] = 0
x[3, 2] = 0
x[3, 3] = 0


Alternativamente...

In [89]:
A = [1 0 0 0 0 0 0 0 0;
0 1 0 0 1 0 0 1 0;
0 0 1 0 0 1 0 0 1]
b = [20; 10; 30]

C = [1 1 1 0 0 0 0 0 0 ;
0 0 0 0 1 1 0 0 0 ;
0 0 0 0 0 0 0 1 1]
d = [50; 50; 20];

LPP_Transporte(i, j, weights, A, C, b, d)

Função Objetivo: 50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 30 x[2,2] + 40 x[2,3] + 60 x[3,2] + 50 x[3,3]
x[1,1] = 20.0
x[1,2] = 0.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 10.0
x[2,3] = 30.0
x[3,1] = 0.0
x[3,2] = 0.0
x[3,3] = 0.0
Valor Objetivo (Custo Mínimo): 2500.0


Observações: Iluminação recebe apenas de eletricidade, e aquecimento ambiente/água recebem apenas de painéis solares, que são a opção mais barata.